# Spin Qubit Pulse Sequences with the HDAWG

This notebook shows you how to use pulse sequences with an HDAWG to realize various experiments. While the sequences are mostly general, they are adapted to those typically used for spin qubits by adding the gate pulsing to control between Coulomb and spin blockade.

# General Imports

In [2]:
%config IPCompleter.greedy=True

# import matplotlib.pyplot as plt
import numpy as np
import time
import zhinst.core
import laboneq
from laboneq.simple import *
import helpers
import matplotlib.pyplot as plt
#install_token("LabOneQ.AllQuantumInOne.")
DO_EMULATION = False # run in emulation mode by default

print(laboneq.__version__)

2.11.0


# Sampling card setup

# Device Setup

Device specific parameters are still required.

## Create device setup

In [3]:
descriptor="""
instrument_list:
  HDAWG:
  - address: DEV8721
    uid: device_hdawg
    interface: usb
connections:
  device_hdawg:
    - rf_signal: q0/fg8_line
      ports: [SIGOUTS/0]
    - rf_signal: q0/fg9_line
      ports: [SIGOUTS/1]
"""
#Change back to to SIGOUTS/2 and SIGOUTS/3
#And change fg8 and fg9
device_setup = DeviceSetup.from_descriptor(
    descriptor,
    server_host="127.0.0.1",
    server_port="8004",
    setup_name="ZI_HDAWG",
)

c:\Users\B15measure2\anaconda3\envs\laboneq\lib\site-packages\laboneq\dsl\device\_device_setup_generator.py:1104: FutureWarning: 'instrument_list' section is deprecated in setup descriptor, use 'instruments' instead.
  warnings.warn(


# S-T readout

## Define parameters for experiment

In [4]:
points_map={
    'level1': [0.2, -0],
    'level2': [-0, 0],}


times_map = {'pulse_time': 5e-3,
             'trigger_time': 1e-3}

In [5]:
# define pulse shapes
@pulse_library.register_pulse_functional
def ramp(x,start=0,stop=1 , **_):
    pulse=start+ (stop-start)*(x+1)/2 # Time normalized to (-1,1)
    return pulse

compress_level_pulse=pulse_library.const(uid="compress_level",length=times_map['pulse_time'],amplitude=1,can_compress=True)


In [6]:
## Create Experiment
exp = Experiment(
    "Singlet-triplet qubit example",
    signals=[
        #ExperimentSignal("gate1"),
        #ExperimentSignal("gate2"),
        ExperimentSignal("gate3"),
        ExperimentSignal("gate4"),
    ],
)

# define experiment
NUM_REP = 50

with exp.acquire_loop_rt(
    uid=("shots"), count=NUM_REP, averaging_mode=AveragingMode.SEQUENTIAL
):
    # with exp.section(
    #     uid="outer_trigger", 
    #     length=times_map['trigger_time'],
    #     alignment=SectionAlignment.LEFT):
    #     exp.reserve(signal="gate3")

    with exp.section(
        uid=("level1"),
        length=times_map['pulse_time'], 
        trigger={"gate3": {"state": 1}}, 
        alignment=SectionAlignment.LEFT,
    ):
        #exp.play(signal="gate1", pulse=compress_level_pulse, amplitude=points_map['level1'][1])
        #exp.play(signal="gate2", pulse=compress_level_pulse, amplitude=points_map['level1'][1])
        exp.play(signal="gate3", pulse=compress_level_pulse, amplitude=points_map['level1'][0])
        exp.play(signal="gate4", pulse=compress_level_pulse, amplitude=points_map['level1'][1])    
    with exp.section(
        uid=("level2"),
        length=times_map['pulse_time'], 
        alignment=SectionAlignment.LEFT,
    ):
        #exp.play(signal="gate1", pulse=compress_level_pulse, amplitude=points_map['level2'][1])
        #exp.play(signal="gate2", pulse=compress_level_pulse, amplitude=points_map['level2'][1])
        exp.play(signal="gate3", pulse=compress_level_pulse, amplitude=points_map['level2'][0])
        exp.play(signal="gate4", pulse=compress_level_pulse, amplitude=points_map['level2'][1])

print(exp.play)  

<bound method Experiment.play of Experiment(uid='Singlet-triplet qubit example', signals={'gate3': ExperimentSignal(uid='gate3', calibration=None, mapped_logical_signal_path=None), 'gate4': ExperimentSignal(uid='gate4', calibration=None, mapped_logical_signal_path=None)}, version=DSLVersion.V3_0_0, epsilon=0.0, sections=[AcquireLoopRt(uid='shots', alignment=SectionAlignment.LEFT, execution_type=ExecutionType.REAL_TIME, length=None, play_after=None, children=[Section(uid='level1', alignment=SectionAlignment.LEFT, execution_type=ExecutionType.REAL_TIME, length=0.005, play_after=None, children=[PlayPulse(signal='gate3', pulse=PulseFunctional(function='const', uid='compress_level', amplitude=1, length=0.005, can_compress=True, pulse_parameters=None), amplitude=0.2, increment_oscillator_phase=None, phase=None, set_oscillator_phase=None, length=None, pulse_parameters=None, precompensation_clear=None, marker=None), PlayPulse(signal='gate4', pulse=PulseFunctional(function='const', uid='compres

In [7]:
# shortcut to the logical signal group q0
lsg = device_setup.logical_signal_groups["q0"].logical_signals

# define signal map
map_signals = {
    "gate3" : lsg["fg8_line"],
    "gate4" : lsg["fg9_line"]
}

In [8]:
# create and connect to session
session = Session(device_setup=device_setup)
session.connect(do_emulation=DO_EMULATION)
# set experiment calibration and signal map
exp.set_signal_map(map_signals)

if not session.connection_state.emulated:
    instrument_serial = device_setup.instrument_by_uid("device_hdawg").address
    device = session.devices[instrument_serial]
    device.triggers.out[2].delay(23.9e-9)

session.run(exp)

2023.08.08 10:58:13.501 laboneq.controller.laboneq_logging INFO   Logging initialized from [Default inline config in laboneq.controller.laboneq_logging] logdir is c:\Users\B15measure2\HDAWG\laboneq_output\log
2023.08.08 10:58:13.504 laboneq.controller.controller  INFO   VERSION: laboneq 2.11.0


2023.08.08 10:58:13.506 laboneq.controller.devices.device_collection INFO   Connecting to data server at 127.0.0.1:8004
2023.08.08 10:58:13.803 laboneq.controller.communication INFO   Connected to Zurich Instruments LabOne Data Server version 23.02 at 127.0.0.1:8004
2023.08.08 10:58:14.062 laboneq.controller.devices.device_collection INFO   Configuring the device setup
2023.08.08 10:58:14.241 laboneq.controller.devices.device_collection INFO   The device setup is configured
2023.08.08 10:58:14.495 laboneq.compiler.workflow.compiler INFO   Starting LabOne Q Compiler run...
2023.08.08 10:58:14.500 laboneq.compiler.scheduler.scheduler INFO   Schedule completed
2023.08.08 10:58:19.553 laboneq.compiler.workflow.compiler INFO   Total seqC lines generated: 32
2023.08.08 10:58:19.555 laboneq.compiler.workflow.compiler INFO   Total sample points generated: 128
2023.08.08 10:58:19.555 laboneq.compiler.workflow.compiler INFO   Finished LabOne Q Compiler run.
2023.08.08 10:58:19.777 laboneq.contro

Results(experiment=Experiment(uid='Singlet-triplet qubit example', signals={'gate3': ExperimentSignal(uid='gate3', calibration=None, mapped_logical_signal_path='/logical_signal_groups/q0/fg8_line'), 'gate4': ExperimentSignal(uid='gate4', calibration=None, mapped_logical_signal_path='/logical_signal_groups/q0/fg9_line')}, version=DSLVersion.V3_0_0, epsilon=0.0, sections=[AcquireLoopRt(uid='shots', alignment=SectionAlignment.LEFT, execution_type=ExecutionType.REAL_TIME, length=None, play_after=None, children=[Section(uid='level1', alignment=SectionAlignment.LEFT, execution_type=ExecutionType.REAL_TIME, length=0.005, play_after=None, children=[PlayPulse(signal='gate3', pulse=PulseFunctional(function='const', uid='compress_level', amplitude=1, length=0.005, can_compress=True, pulse_parameters=None), amplitude=0.2, increment_oscillator_phase=None, phase=None, set_oscillator_phase=None, length=None, pulse_parameters=None, precompensation_clear=None, marker=None), PlayPulse(signal='gate4', pu

### View experiment in pulse sheet viewer

To-do: 
1. Add stability diagram visualization

In [9]:
# use pulse sheet viewer to display the pulse sequence - only recommended for small number of averages and sweep steps to avoid performance issues
compiled_exp = session.compiled_experiment
show_pulse_sheet(".\laboneq_output\ST pulse test", compiled_exp)

2023.08.08 10:58:36.432 laboneq.pulse_sheet_viewer.pulse_sheet_viewer INFO   Writing html file to c:\Users\B15measure2\HDAWG\laboneq_output\ST pulse test_2023-08-08-10-58-36.html


c:\Users\B15measure2\HDAWG\laboneq_output\ST pulse test_2023-08-08-10-58-36.html

In [10]:
helpers.interactive_psv(compiled_exp)

# 2D sweep example, wait time vs phase of second pulse

## Experiment

In [ ]:
## constant definition
X90_DURATION = 10e-9  # [s]

START = 0
STOP = 2 * np.pi
STEPS = 5
NUM_REP = 2

START_DELAY = 0
STOP_DELAY = LEN_COULOMB_CYCLE / 2 - 2 * X90_DURATION
STEPS_DELAY = 3


In [ ]:
## define phase sweep parameter
sweep_phase = LinearSweepParameter(
    uid="phase_sweep", start=START, stop=STOP, count=STEPS
)
sweep_delay = LinearSweepParameter(
    uid="Ramsey_delay", start=START_DELAY, stop=STOP_DELAY, count=STEPS_DELAY
)

print(sweep_phase.values / np.pi)

In [ ]:
## Create Experiment
exp = Experiment(
    "Ramsey variant I",
    signals=[
        ExperimentSignal("drive"),
        ExperimentSignal("gate1"),
        ExperimentSignal("gate2"),
    ],
)

# define experiment
with exp.acquire_loop_rt(
    uid="shots", count=NUM_REP, averaging_mode=AveragingMode.CYCLIC
):
    with exp.sweep(uid="sweep_delay", parameter=sweep_delay):
        with exp.sweep(uid="sweep_phase", parameter=sweep_phase):
            with exp.section(
                uid="qubit_excitation",
                alignment=SectionAlignment.RIGHT,
                length=LEN_COULOMB_CYCLE,
            ):
                exp.play(signal="gate1", pulse=coulomb_pulse, amplitude=0.5)
                exp.play(signal="gate1", pulse=coulomb_pulse, amplitude=0.75)

                exp.play(signal="gate2", pulse=coulomb_pulse, amplitude=0.5)
                exp.play(signal="gate2", pulse=coulomb_pulse, amplitude=0.75)

                exp.play(signal="drive", pulse=drive_pulse, set_oscillator_phase=0)
                exp.delay(signal="drive", time=sweep_delay)
                exp.play(
                    signal="drive",
                    pulse=drive_pulse,
                    increment_oscillator_phase=sweep_phase,
                )
            with exp.section(
                uid="qubit_readout",
                alignment=SectionAlignment.RIGHT,
                length=LEN_READOUT,
            ):
                exp.play(signal="gate1", pulse=coulomb_readout, amplitude=0.3)
                exp.play(signal="gate2", pulse=coulomb_readout, amplitude=0.3)
            with exp.section(
                uid="outer_trigger",
                length=LEN_READOUT,
                trigger={"drive": {"state": 1}},
                alignment=SectionAlignment.RIGHT,
            ):
                with exp.section(
                    uid="inner_trigger",
                    length=LEN_READOUT - 100e-9,
                    trigger={"drive": {"state": 2}},
                ):
                    exp.reserve(signal="drive")


In [ ]:
# create and connect to session
session = Session(device_setup=device_setup)
session.connect(do_emulation=DO_EMULATION)

# set experiment calibration and signal map
exp.set_calibration(calib_q0)
exp.set_signal_map(map_q0)

session.run(exp)

## View experiment in pulse sheet viewer

In [ ]:
# use pulse sheet viewer to display the pulse sequence - only recommended for small number of averages and sweep steps to avoid performance issues
compiled_exp = session.compiled_experiment
show_pulse_sheet("2D example", compiled_exp)

